# Stock Prediction with Support Vector Regressor

## Import Libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [4]:
import datetime
from pandas_datareader import data, wb

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
import warnings
warnings.filterwarnings("ignore") 

## Collecting Data

In [7]:
start_date = datetime.datetime(2010, 1, 1)
end_date = datetime.datetime.today()         

#### Create DataFrame

In [8]:
SBI = data.DataReader("SBIN.NS", data_source='yahoo', start=start_date, end=end_date)
HDFC = data.DataReader("HDFCBANK.NS", data_source='yahoo', start=start_date, end=end_date)
AXIS = data.DataReader("AXISBANK.NS", data_source='yahoo', start=start_date, end=end_date)
BOB = data.DataReader("BANKBARODA.NS", data_source='yahoo', start=start_date, end=end_date)
PNB = data.DataReader("PNB.NS", data_source='yahoo', start=start_date, end=end_date)
ICICI = data.DataReader("ICICIBANK.NS", data_source='yahoo', start=start_date, end=end_date)
YESBANK = data.DataReader("YESBANK.NS", data_source='yahoo', start=start_date, end=end_date)
KOTAK = data.DataReader("KOTAKBANK.NS", data_source='yahoo', start=start_date, end=end_date)

## Getting Data Ready

In [9]:
# Get the Adj Close Price
sbi_df = SBI[["Adj Close"]]
hdfc_df = HDFC[["Adj Close"]]
axis_df = AXIS[["Adj Close"]]
bob_df = BOB[["Adj Close"]]
pnb_df = PNB[["Adj Close"]]
icici_df = ICICI[["Adj Close"]]
yesbank_df = YESBANK[["Adj Close"]]
kotak_df = KOTAK[["Adj Close"]]

In [10]:
# A variable for predicting 'n' days out into the future 
forecast_out = 30

# Create another column (the target or dependent variable) shifted 'n' units up
sbi_df['Prediction'] = sbi_df[['Adj Close']].shift(-forecast_out)
hdfc_df['Prediction'] = hdfc_df[['Adj Close']].shift(-forecast_out)
axis_df['Prediction'] = axis_df[['Adj Close']].shift(-forecast_out)
bob_df['Prediction'] = bob_df[['Adj Close']].shift(-forecast_out)
pnb_df['Prediction'] = pnb_df[['Adj Close']].shift(-forecast_out)
icici_df['Prediction'] = icici_df[['Adj Close']].shift(-forecast_out)
yesbank_df['Prediction'] = yesbank_df[['Adj Close']].shift(-forecast_out)
kotak_df['Prediction'] = kotak_df[['Adj Close']].shift(-forecast_out)

In [11]:
# List of the ticker symbols (as strings) in alphabetical order
tickers = 'SBI HDFCBANK AXISBANK BANKBARODA PNB ICICIBANK YESBANK KOTAKBANK'.split()
tickers.sort()

In [12]:
# Create a Single DataFrame by Concatenating all the above DataFrames
bank_stocks = pd.concat([axis_df, bob_df, hdfc_df, icici_df, kotak_df, pnb_df, sbi_df, yesbank_df], axis=1, keys=tickers)
bank_stocks = bank_stocks[:-30]
bank_stocks.head()

AXISBANK             BANKBARODA               HDFCBANK  \
             Adj Close  Prediction  Adj Close Prediction   Adj Close   
Date                                                                   
2010-01-04  140.712021  151.377823  51.206989  57.293846  134.245209   
2010-01-05  143.647934  155.802994  52.686508  58.495327  134.363205   
2010-01-06  141.236801  155.235626  53.009216  56.608700  134.453751   
2010-01-07  142.236725  157.072403  53.326965  56.161865  134.803970   
2010-01-08  144.094727  155.753357  53.540455  56.971127  134.981079   

                        ICICIBANK              KOTAKBANK              \
            Prediction  Adj Close Prediction   Adj Close  Prediction   
Date                                                                   
2010-01-04  130.329666  85.319977  81.561913  202.426041  186.522232   
2010-01-05  132.517624  86.130264  81.474457  204.430389  184.850494   
2010-01-06  133.316467  86.789574  80.844452  206.915863  179.573486   
2010-01-07  133.528961  85.970222  80.655098  209.240005  176.166992   
2010-01-08  133.875275  84.762535  82.221260  207.948059  173.558487   

                  PNB                   SBI               YESBANK             
            Adj Close Prediction  Adj Close Prediction  Adj Close Prediction  
Date                                                                          
2010-01-04  84.943443  86.630409  95.066895  85.587837  34.458496  31.044094  
2010-01-05  86.255913  87.150787  95.102180  84.955849  34.965523  30.293184  
2010-01-06  85.847481  86.114899  95.672699  83.333183  34.471333  29.439579  
2010-01-07  84.668106  85.934959  95.137444  83.816475  33.887291  30.107061  
2010-01-08  84.792007  85.910637  94.853218  83.621841  33.726841  30.312439

# Creating and Evaluating the Model

In [13]:
svr_accuracy = {}

for i in range(len(tickers)):
    
    bank = bank_stocks[tickers[i]]
    
    # X and y
    X = np.array(bank.drop(['Prediction'], axis=1))
    y = np.array(bank["Prediction"])
    
    # Training and Testing Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    svr = SVR(kernel='rbf', C=1e3, gamma=0.1)
    
    svr.fit(X_train, y_train)
    
    svr_confidence = svr.score(X_test, y_test)
    svr_accuracy[tickers[i]] = svr_confidence
    print(tickers[i])
    print("Support Vector Machine: {:.3f}%".format(svr_confidence*100))
    print()

AXISBANK
Support Vector Machine: 92.594%

BANKBARODA
Support Vector Machine: 84.509%

HDFCBANK
Support Vector Machine: 97.240%

ICICIBANK
Support Vector Machine: 89.221%

KOTAKBANK
Support Vector Machine: 94.333%

PNB
Support Vector Machine: 80.638%

SBI
Support Vector Machine: 84.419%

YESBANK
Support Vector Machine: 92.701%

